# Chapter 5

In [13]:
# Import create_engine, MetaData
from sqlalchemy import create_engine, MetaData

# Define an engine to connect to chapter5.sqlite: engine
engine = create_engine('sqlite:///chapter5.sqlite')

# Initialize MetaData: metadata
metadata = MetaData()


In [14]:
# Import Table, Column, String, and Integer
from sqlalchemy import Table, Column, String, Integer

# Build a census table: census
census = Table('census', metadata,
               Column('state', String(30)),
               Column('sex', String(1)),
               Column('age', Integer()),
               Column('pop2000', Integer()),
               Column('pop2008', Integer()))

# Create the table in the database
metadata.create_all(engine)


In [15]:
import csv
file = open('census.csv')
csv_reader = csv.reader(file)

In [16]:
# Create an empty list: values_list
values_list = []

# Iterate over the rows
for row in csv_reader:
    # Create a dictionary with the values
    data = {'state': row[0], 'sex': row[1], 'age':row[2], 'pop2000': row[3],
            'pop2008': row[4]}
    # Append the dictionary to the values list
    values_list.append(data)


In [17]:
len(values_list)

8772

In [18]:
# data prep
connection = engine.connect()

In [19]:
# Import insert
from sqlalchemy import insert

# Build insert statement: stmt
stmt = insert(census)

# Use values_list to insert data: results
results = connection.execute(stmt, values_list)

# Print rowcount
print(results.rowcount)


8772


In [22]:
from sqlalchemy import func

In [27]:
# Import select
from sqlalchemy import select

# Calculate weighted average age: stmt
stmt = select([census.columns.sex,
               (func.sum(census.columns.pop2008 * census.columns.age) /
                func.sum(census.columns.pop2008)).label('average_age')
               ])

# Group by sex
stmt = stmt.group_by(census.columns.sex)

# Execute the query and store the results: results
results = connection.execute(stmt).fetchall()

# Print the average age by sex
for row in results:
    print(row[0], row[1])


F 38
M 35


In [33]:
# import case, cast and Float from sqlalchemy
from sqlalchemy import case, cast, Float

# Build a query to calculate the percentage of females in 2000: stmt
stmt = select([census.columns.state,
    (func.sum(
        case([
            (census.columns.sex == 'F', census.columns.pop2000)
        ], else_=0)) /
     cast(func.sum(census.columns.pop2000), Float) * 100).label('percent_female')
])

# Group By state
stmt = stmt.group_by(census.columns.state)

# Execute the query and store the results: results
results = connection.execute(stmt).fetchall()

# Print the percentage
for result in results:
    print(result.state, result.percent_female)


Alabama 51.832407770179465
Alaska 49.301497893484594
Arizona 50.22361303057914
Arkansas 51.26992846221834
California 50.35233214901979
Colorado 49.84767060299562
Connecticut 51.66816507130644
Delaware 51.61109733558627
District of Columbia 53.129626141738385
Florida 51.36488001165242
Georgia 51.11408350339436
Hawaii 51.118011836915514
Idaho 49.98972623903102
Illinois 51.11224234802867
Indiana 50.95480313297678
Iowa 50.950398342534264
Kansas 50.821864107754735
Kentucky 51.32687036927168
Louisiana 51.75351596554121
Maine 51.50570813418951
Maryland 51.93575549972231
Massachusetts 51.843023571316785
Michigan 50.97246518318712
Minnesota 50.49332944301148
Mississippi 51.92229481794672
Missouri 51.46888602639692
Montana 50.32202690728538
Nebraska 50.8584549336086
Nevada 49.36736361384359
New Hampshire 50.858019844961746
New Jersey 51.51713956125773
New Mexico 51.0471720798335
New York 51.83453865150073
North Carolina 51.482262322084594
North Dakota 50.50069363231332
Ohio 51.46550350015544
Okl

In [42]:
# data prep
from sqlalchemy import desc

In [43]:
# Build query to return state name and population difference from 2008 to 2000
stmt = select([census.columns.state,
     (census.columns.pop2008 - census.columns.pop2000).label('pop_change')
])

# Group by State
stmt = stmt.group_by(census.columns.state)

# Order by Population Change
stmt = stmt.order_by(desc('pop_change'))

# Limit to top 10
stmt = stmt.limit(10)

# Use connection to execute the statement and fetch all results
results = connection.execute(stmt).fetchall()

# Print the state and population change for each record
for result in results:
    print('{}:{}'.format(result.state, result.pop_change))


Texas:40137
California:35406
Florida:21954
Arizona:14377
Georgia:13357
North Carolina:11574
Virginia:6639
Colorado:6425
Utah:5934
Illinois:5412
